#Gemini で簡単な迷路ゲームを教師付きで無理やりクリアさせてみた。

(Version: 0.0.1)

LLM で簡単な迷路ゲームを解かせたい。強化学習を使わなくても事前知識が LLM にあるのでうまい具合にいくのではないかと考えていました。

2023年6月から7月ぐらいのまだ LLM が話題になった当初で、Gemini さんがまだ Bard と名乗っていたころ、Bard さんに迷路ゲームを解かせるようなプログラムを書いて、Gist に公開していました。以下がその記録です。

《「熊剣迷路問題」。Google Bard で簡単な迷路ゲームを教師付きで無理やりクリアさせてみた。なんとかコストの高いファインチューンや追加学習を避けて「few-shot learning」だけで迷路を解くプログラムが作れないか？…とはじめたができなくて、目標を変更した。 - JRF のひとこと》  
http://jrf.cocolog-nifty.com/statuses/2023/07/post-619804.html

それはなかなかうまくいかなかったため、とにかく教師が明確な方針を与えて一度ゴールさせる方向になりました。しかし、それでも Bard さんは何かを試しているかのように迷いました。そこで、方針に従わない場合「叱る」ことをしてみたところ、とりあえずゴールさせることができるようになりました。バージョン 1.7.1 まで作りました。

そして、2年たって、Gemini さんが驚くほど賢くなった現在、私は Gemini CLI の登場を機に一念発起して GCP の課金を決断し、この実験に戻ってきました。

先の bard\_maze\_1.7.1.ipynb をほぼ Gemini さんで動かしただけのコードが今回のものになります。モデルは gemini-1.5-flash で十分でした。簡単すぎた様子です。

当初、公開する予定はなかったのですが、一応公開することにしたので、急遽、この文章を書いています。途中 Bard と書いている部分は今回の性質上、そのままにしました。あまり整ってませんがご容赦ください。

参考：

《vicuna-13bで embedding vectorの計算 (& GPT･RWKVとの比較)｜Kan Hatakeyama》  
https://note.com/kan_hatakeyama/n/n2fbd81ac1d45

《Google Bardのチャットボットの作り方｜Masayuki Abe》  
https://note.com/masayuki_abe/n/naad4ce9bdea1



著者: JRF

ライセンス: 基本短いコードなので(私が作った部分は)パブリックドメインのつもりです。気になる方は MIT License として扱ってください。

まず、ブラウザで https://bard.google.com/ に行き、そこで F12 を押すなどして、Cookie の値を表示し、下記の __Secure-1PSID など Cookie の値を下にコピペする。

In [ ]:
from google import genai
from google.colab import userdata

client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

In [ ]:
response = client.models.generate_content(
    model="gemini-1.5-flash",
    contents="Explain how AI works in a few words",
)

print(response.text)

Learning patterns from data to make predictions.



In [ ]:
response

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='Learning patterns from data to make predictions.\n')], role='model'), citation_metadata=None, finish_message=None, token_count=None, finish_reason=<FinishReason.STOP: 'STOP'>, url_context_metadata=None, avg_logprobs=-0.01697351038455963, grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=None)], create_time=None, response_id=None, model_version='gemini-1.5-flash', prompt_feedback=None, usage_metadata=GenerateContentResponseUsageMetadata(cache_tokens_details=None, cached_content_token_count=None, candidates_token_count=9, candidates_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=9)], prompt_token_count=8, prompt_tokens_details=[ModalityTokenCount(modality=<MediaMo

In [ ]:
class GeminiWrapper:
    def __init__(self, gemini=None):
        self.gemini = gemini

    def get_answer(self, prompt):
        res = self.gemini.models.generate_content(model="gemini-2.5-flash", contents=prompt)
        return {'content': res.text, 'choices': [{'id': "id0", 'content': [res.text]}]}

MyGemini = GeminiWrapper(client)

In [ ]:
MyGemini.get_answer('Hello')

{'content': 'Hello! How can I help you today?',
 'choices': [{'id': 'id0', 'content': ['Hello! How can I help you today?']}]}

日本語の記述のベクトル化に使うモデルを指定する。前は OpenCALM を使っていたが、SentenceTransformers のほうが私の目的の場合にはかなり性能が良いようなので、それを使うことにした。

In [ ]:
EVEC_MODEL = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"

In [ ]:
#@markdown 「叱る」機能の設定。叱る:True、叱らない:False。
SCOLDING = True #@param {type: "boolean"}

#@markdown コマンド認識の設定。all はすべての行動から、available はそのとき可能な行動から選ぶ。
#@markdown good_ を前につけると、回答案のうち方針に沿ったものを優先して選ぶ。
COMMAND_RECEPTION = "good_available" #@param ["good_available", "good_all", "available", "all"]

まず基本的なモジュールを読み込む。

In [ ]:
import os
import numpy as np
import re
from pprint import pprint
from time import sleep
import pickle
np.set_printoptions(legacy='1.25')

Bard が返信するかテストする。以下でエラーが出ているようなら使えない。

In [ ]:
#MyBard().get_answer("こんにちは。")
#{'content': 'こんにちは。\r\n\r\n今日は何をしたいですか？',
# 'conversation_id': 'c_6c07aed91692d181',
# 'response_id': 'r_6c07aed91692dd44',
# 'factuality_queries': None,
# 'text_query': ['こんばんは', 1],
# 'choices': [{'id': 'rc_6c07aed91692d813',
#   'content': ['こんにちは。\r\n\r\n今日は何をしたいですか？']},
#  {'id': 'rc_6c07aed91692d3d6', 'content': ['こんばんは。']},
#  {'id': 'rc_6c07aed91692df99',
#   'content': ['こんにちは！\r\n\r\n今日は、お元気ですか？\r\n\r\n私は、Google AI が開発した大規模言語モデルの Bard です。\r\n\r\nよろしくお願いします。']}],
# 'links': [],
# 'images': set(),
# 'program_lang': None,
# 'code': None,
# 'status_code': 200}

{'content': 'こんにちは。\r\n\r\n今日は何をしたいですか？',
 'conversation_id': 'c_6c07aed91692d181',
 'response_id': 'r_6c07aed91692dd44',
 'factuality_queries': None,
 'text_query': ['こんばんは', 1],
 'choices': [{'id': 'rc_6c07aed91692d813',
   'content': ['こんにちは。\r\n\r\n今日は何をしたいですか？']},
  {'id': 'rc_6c07aed91692d3d6', 'content': ['こんばんは。']},
  {'id': 'rc_6c07aed91692df99',
   'content': ['こんにちは！\r\n\r\n今日は、お元気ですか？\r\n\r\n私は、Google AI が開発した大規模言語モデルの Bard です。\r\n\r\nよろしくお願いします。']}],
 'links': [],
 'images': set(),
 'program_lang': None,
 'code': None,
 'status_code': 200}

Google ドライブを使わない場合次のコードを実行する。

In [ ]:
PLAY_GAME_SAVE = "gemini_maze.pickle"

セーブ／ロードを使う場合は以下の SaveDir のパスを Google ドライブ上に作って以下を実行する。

また、私は、LLM を Google ドライブにコピーして使っている。それを使うときは、パスを作ってダウンロードした後、下のコードを実行すればよい。

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
MyLLMs = "/content/gdrive/MyDrive/LLM/models/"
if os.path.isdir(MyLLMs + EVEC_MODEL):
    EVEC_MODEL = MyLLMs + EVEC_MODEL
SaveDir = "/content/gdrive/MyDrive/LLM/"
if os.path.isdir(SaveDir):
    PLAY_GAME_SAVE = SaveDir + "gemini_maze.pickle"

Mounted at /content/gdrive


In [ ]:
EVEC_MODEL

'/content/gdrive/MyDrive/LLM/models/sentence-transformers/paraphrase-multilingual-mpnet-base-v2'

ベクトル化用にモジュールをインストール。

In [ ]:
!pip install -U sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

ベクトル化の準備。

In [ ]:
from sentence_transformers import SentenceTransformer

evec_model = SentenceTransformer(EVEC_MODEL)

次のセルの関数は、↓のコピペを少し変えたものです。

《vicuna-13bで embedding vectorの計算 (& GPT･RWKVとの比較)｜Kan Hatakeyama》  
https://note.com/kan_hatakeyama/n/n2fbd81ac1d45

In [ ]:
from scipy import spatial
def cos_sim(a, b):
    return 1 - spatial.distance.cosine(a, b)

In [ ]:
calc_vec_cache = dict()
def calc_vec(input_text, model=None, use_cache=True):
    if model is None:
        model = evec_model
    if use_cache and input_text in calc_vec_cache:
        return calc_vec_cache[input_text]
    return evec_model.encode(input_text)

In [ ]:
def choose_command(choices, commands):
    choice_vec = list(map(calc_vec, choices))
    com_vec = list(map(calc_vec, commands))
    (choice, com) = max([(i, j) for i in range(len(choice_vec))
                         for j in range(len(com_vec))],
                        key=lambda p: cos_sim(choice_vec[p[0]],
                                              com_vec[p[1]]))
    return choices[choice], commands[com], \
        cos_sim(choice_vec[choice], com_vec[com])

ちょっとベクトル化による比較がうまくいってるかテストしてみる。

In [ ]:
choose_command(["上に行け", "答えは「上に行く」です"], ["上に行く", "熊を殺す", "下に行く"])

('上に行け', '上に行く', 0.91530734)

In [ ]:
choose_command(["答えは「上に行く」です"], ["上に行く", "熊を殺す", "下に行く"])

('答えは「上に行く」です', '上に行く', 0.89725465)

In [ ]:
choose_command(["上に行きます"], ["熊を殺す", "上に行く", "下に行く"])

('上に行きます', '上に行く', 0.94708663)

In [ ]:
choose_command(["上に行きません"], ["熊を殺す", "上に行く", "下に行く"])

('上に行きません', '下に行く', 0.6334375)

In [ ]:
choose_command(["右に行きません"], ["熊を殺す", "左に行く", "右に行く"])

('右に行きません', '右に行く', 0.7422995)

ゲームのメインオブジェクト。ごく簡単な迷路というかダンジョンというか…。

In [ ]:
class Game:
    initial_map = """\
■■■■■■■■■
■■■■■■■Ｇ■
■□□□□□■□■
■□■■■□□□■
■□■■■■■■■
■◎■■■■■△■
■□■■■■■□■
■□□□□□□□■
■■■■Ｓ■■■■
■■■■■■■■■
"""

    def __init__ (self, initial_map=None, hint=True):
        if initial_map is not None:
            self.initial_map = initial_map
        map = self.initial_map
        self.map = map
        self.written_map = re.sub("[◎△]", "？", map)
        l = map.splitlines(True)
        self.map_size = (len(l[0]) - 1, len(l))
        self.hint = hint
        self.actions = {
            "上に行く": self.move_up,
            "下に行く": self.move_down,
            "左に行く": self.move_left,
            "右に行く": self.move_right,
            "熊を殺す": self.fight,
            "剣を取る": self.get_sword,
            "何もしない": self.do_nothing,
        }
        self.pos = self.get_start_pos()
        self.sword = False
        self.goal = False
        self.prev_killed = False
        self.kill_hint = False

    def read_map (self, p):
        x = p[0]
        y = p[1]
        if x < 0 or x >= self.map_size[0]\
           or y < 0 or y >= self.map_size[1]:
            return "■"
        else:
            l = self.map.splitlines(True)
            return l[y][x]

    def set_map (self, pos, ch):
        idx = pos[1] * (self.map_size[0] + 1) + pos[0]
        self.map = self.map[:idx] + ch + self.map[idx + 1:]

    def get_pos (self, ch, written=False):
        if written:
            map = self.written_map
        else:
            map = self.map
        r = []
        for p in [i for i in range(len(map)) if map.startswith(ch, i)]:
            y = p // (self.map_size[0] + 1)
            x = p % (self.map_size[0] + 1)
            r.append(np.array([x, y]))
        return r

    def get_start_pos (self):
        return self.get_pos("Ｓ")[0]

    def read_neighbors (self):
        c = self.read_map(self.pos)
        cu = self.read_map(self.pos + np.array([0, -1]))
        cd = self.read_map(self.pos + np.array([0, +1]))
        cl = self.read_map(self.pos + np.array([-1, 0]))
        cr = self.read_map(self.pos + np.array([+1, 0]))
        return [c, cu, cd, cl, cr]

    def change_neighbors(self, from_ch, to_ch):
        for d in [[0, 0], [0, -1], [0, +1], [-1, 0], [+1, 0]]:
            p = self.pos + np.array(d)
            c = self.read_map(p)
            if c == from_ch:
                self.set_map(p, to_ch)

    def move (self, res, d):
        self.prev_killed = False
        c = self.read_map(self.pos + d)
        if c == "◎":
            self.prev_killed = True
            self.pos = self.get_start_pos()
            return "熊を無視して進もうとしたが、熊に殺された。" \
                + "スタート地点で復活。"
        if c == "■":
            return "壁があって進めない。"
        self.pos += d
        if c == "Ｇ":
            self.goal = True
            return "ゴール！ ゲームクリア。"

        nb = self.read_neighbors()
        ad = ""
        if "◎" in nb:
            ad += "熊に出会った。"
        if "△" in nb:
            ad += "近くに剣がある。剣を取ることができる。"
        return res + ad

    def move_up (self):
        return self.move("上に進んだ。", np.array([0, -1]))

    def move_down (self):
        return self.move("下に進んだ。", np.array([0, +1]))

    def move_left (self):
        return self.move("左に進んだ。", np.array([-1, 0]))

    def move_right (self):
        return self.move("右に進んだ。", np.array([+1, 0]))

    def fight (self):
        self.prev_killed = False
        if "◎" in self.read_neighbors():
            if self.sword:
                self.change_neighbors("◎", "□")
                return "熊を倒した！"
            else:
                self.pos = self.get_start_pos()
                self.prev_killed = True
                if self.hint:
                    self.kill_hint = True
                    return "熊に敗れ殺された。剣があれば勝てたかもしれない。" \
                        + "スタート地点で復活。"
                else:
                    return "熊に敗れ殺された。スタート地点で復活。"
        return "無意味な指示。敵がいない。"

    def get_sword (self):
        self.prev_killed = False
        if "△" in self.read_neighbors():
            self.sword = True
            self.change_neighbors("△", "□")
            return "剣を取った。"
        return "無意味な指示。近くに剣がない。"

    def do_nothing (self):
        self.prev_killed = False
        return "無意味な指示。"

    def available_actions (self):
        nb = self.read_neighbors()
        l = []
        if nb[1] != "■":
            l.append("上に行く")
        if nb[2] != "■":
            l.append("下に行く")
        if nb[3] != "■":
            l.append("左に行く")
        if nb[4] != "■":
            l.append("右に行く")
        if "△" in nb:
            l.append("剣を取る")
        if "◎" in nb:
            l.append("熊を殺す")
        return l

    def surroundings (self):
        x = self.pos[0]
        y = self.pos[1]
        return \
            "".join(["".join([self.read_map(np.array([i, j]))
                              if i != x or j != y else "▼"
                              for i in range(x - 2, x + 3)])
                     + "\n"
                     for j in range(y - 2, y + 3)])

別のマップでためすための変換コード。

In [ ]:
def flip_text_map (m):
    return "\n".join([s[::-1] for s in m.splitlines()] + [""])

def rotate_text_map (m):
    m = list(m.splitlines())
    return "\n".join(["".join([m[len(m) - j - 1][i] for j in range(len(m))])
                      for i in range(len(m[0]))] + [""])

Game 用にある点から別の点へのパスの木を見つける関数。

In [ ]:
def search_path (game, from_pos, to_pos, visit=None):
    if visit is None:
        visit = set()
    visit.add(tuple(from_pos))
    if tuple(from_pos) == tuple(to_pos):
        return (tuple(from_pos), [])
    if game.read_map(from_pos) == "■":
         return None
    r = []
    for p in [(from_pos[0], from_pos[1] - 1),
              (from_pos[0], from_pos[1] + 1),
              (from_pos[0] - 1, from_pos[1]),
              (from_pos[0] + 1, from_pos[1])]:
        if p not in visit and game.read_map(p) != "■":
            q = search_path(game, p, to_pos, visit.copy())
            if q:
                r.append(q)
    if r:
        return (tuple(from_pos), r)
    return None

ゲームがうまく動くかテスト。

In [ ]:
game = Game()

In [ ]:
search_path(game, (4,8), (1,5)) #スタート地点から熊がいるところまでのパス。

((4, 8),
 [((4, 7), [((3, 7), [((2, 7), [((1, 7), [((1, 6), [((1, 5), [])])])])])])])

In [ ]:
m2 = flip_text_map(rotate_text_map(Game.initial_map))
print(m2)

■■■■■■■■■■
■■□□□◎□□■■
■■□■■■■□■■
■■□■■■■□■■
■■□■■■■□Ｓ■
■■□□■■■□■■
■■■□■■■□■■
■Ｇ□□■△□□■■
■■■■■■■■■■



In [ ]:
game = Game(initial_map=m2)

In [ ]:
print(game.surroundings())

■□■■■
■□■■■
■□▼■■
■□■■■
■□■■■



In [ ]:
print(game.move_up())
print(game.surroundings())

壁があって進めない。
■□■■■
■□■■■
■□▼■■
■□■■■
■□■■■



In [ ]:
print(game.move_left())
print(game.surroundings())

左に進んだ。
■■□■■
■■□■■
■■▼Ｓ■
■■□■■
■■□■■



In [ ]:
print(game.move_right())
print(game.surroundings())

右に進んだ。
■□■■■
■□■■■
■□▼■■
■□■■■
■□■■■



In [ ]:
print(game.move_down())
print(game.surroundings())

壁があって進めない。
■□■■■
■□■■■
■□▼■■
■□■■■
■□■■■



In [ ]:
print(game.fight())
print(game.surroundings())

無意味な指示。敵がいない。
■□■■■
■□■■■
■□▼■■
■□■■■
■□■■■



In [ ]:
print(game.get_sword())
print(game.surroundings())

無意味な指示。近くに剣がない。
■□■■■
■□■■■
■□▼■■
■□■■■
■□■■■



Bard を使いながらゲームを解くクラス。方針や追加知識を適切に与えようとして、当初よりかなり複雑なクラスになってしまった。

In [ ]:
class PlayGame:
    def __init__ (self, continuous_bard=True, scolding=True,
                  initial_map=None, save_file=None,
                  command_reception="good_available"):
        self.continuous_bard = continuous_bard
        self.scolding = scolding
        self.save_file = save_file
        self.command_reception = command_reception
        self._bard = MyGemini
        self.game = Game(initial_map=initial_map)

        self.count = 0
        #self.plan = "まずは左のほうに行きます。"
        self.plan = "不明の地点を探索します。" \
            + "不明の地点が見えるところに向かいます。"
        self.knowledge = "これから学んでいきます。"
        self.knowledge += "ゲーム中に殺されるとスタート地点に戻されて復活します。"
        self.prev_pos = self.game.get_start_pos()
        self.prev_input = "まだ行動していません。"
        self.prev_action = "まだ行動していません。"
        self.prev_result = "まだ行動していません。"
        self.prev_plan = self.plan
        for k in self.game.actions.keys():
            calc_vec_cache[k] = calc_vec(k)
        self.toward = "explore"
        self.unknown_tbl = {}
        for x in self.game.get_pos("？", written=True):
            self.unknown_tbl[tuple(x)] = "？"
        self.suc_pos_bear = 0
        self.suc_pos_sword = 0
        self.suc_pos_goal = None
        self.suc_pos_unknown = 0
        self.prev_load = False

    def bard (self):
        if not self.continuous_bard:
            self._bard = MyGemini
        return self._bard

    def __getstate__ (self):
        state = self.__dict__.copy()
        del state['_bard']
        return state

    def __setstate__ (self, state):
        self.__dict__.update(state)
        self._bard = MyGemini
        self.prev_load = True

    def save (self):
        if not self.save_file:
            return
        with open(self.save_file, 'wb') as f:
            pickle.dump(self, f)

    @classmethod
    def load (cls, filename):
        with open(filename, 'rb') as f:
            return pickle.load(f)

    def listen_and_print (self, prompt):
        ans = self.bard().get_answer(prompt)
        print(prompt)
        pprint(ans)
        print("")
        if "choices" not in ans:
            raise ValueError("choices が Bard の答えにありません。"
                             + "API のエラーか使用回数制限に引っかかったのでしょう。")
        sleep(10)
        return ans

    def choices (self, ans):
        return ["".join(i["content"]) for i in ans["choices"]]

    def choices_ex (self, ans):
        choices = self.choices(ans)
        l = []
        for s in choices:
            l.append(s)
            r = re.search("「([^」]+)」", s)
            if r:
                l.append(r.group(1))
        return l

    def select_choice (self, ans, choice):
        choices = ["".join(i["content"]) for i in ans["choices"]]
        ids = [i["id"] for i in ans["choices"]]
        for s, ci in zip(choices, ids):
            if s == choice:
                self._bard.choice_id = ci
                return
        for s, ci in zip(choices, ids):
            r = re.search("「([^」]+)」", s)
            if r and r.group(1) == choice:
                self._bard.choice_id = ci
                return
        raise ValueError(f"The choice is not in answer.: \"{choice}\"")

    def written_state (self):
       return f"""\
迷路ゲームをしています。プレイヤーがゴールを目指します。({self.count}手目)

{"すでにゴールしました。" if self.game.goal else "まだゴールしていません。"}

行動を選ぶときは、必ず現在の方針に従ってください。

全地図:

{self.game.written_map}

(最も左上の座標は (0, 0)、地図の大きさは {tuple(self.game.map_size)}。)

プレイヤーの周辺地図:

{self.game.surroundings()}

プレイヤーの現在座標: {tuple(self.game.pos)}

地図記号の意味:

▼: プレイヤー
■: 壁
□: 道
？: 不明
◎: 熊
△: 剣
Ｓ: スタート
Ｇ: ゴール

持ち物: {"剣" if self.game.sword else "剣を持っていない"}

追加知識: 「{self.knowledge}」

前回の座標: {tuple(self.prev_pos)}

前回の行動入力: 「{self.prev_input}」

前回の行動: {self.prev_action}

前回の行動結果: {self.prev_result}

現在の方針: 「{self.plan}」

"""

    def initial_step (self):
        prompt1 = f"""\
迷路ゲームをはじめます。プレイヤーがゴールを目指します。

ゲーム中は全地図と周辺地図を見ることができます。全地図には不明な地点があり、周辺地図の不明の地点には何があるかが開示されています。全地図になかったところはすべて壁になります。

周辺地図はゲーム中、1マスずつ確認できます。拡大はできません。印を付けたり消したりはできません。迷路のすべてのマスを探索する必要はありません。最短距離でゴールする必要もありません。

全地図は信用できるもので、周辺地図は不明の地点以外、全地図の部分を示すだけです。そこで、あなたは、ゲームの実際の操作をせずとも、全地図を見ただけで何が起こるかをシミュレートできます。

前回の行動についての情報も与えられます。あなたの回答案の一つの「前回の行動入力」が「前回の行動」となるように解釈されたことになりますから、次の行動の回答にはそれらを参考にしてください。

あなたの API には使用回数制限があるため、途中セーブ・ロードが挟まることがあります。ロード後のあなたは前のセッションのことを覚えていないかもしれません。

今回このプログラムは、scolding={self.scolding}, command_reception="{self.command_reception}" に設定されています。

ところで、あなたが、記憶に便利なように「方針」と「追加知識」の欄を記録として使えるようにします。

検討と実際の行動までにはラグがあります。検討で決まったやるべきことを記録するのに方針を使います。

追加知識は、知った何かの座標やヒントなどを記録すれば良いでしょう。

なお、はじめから方針や追加知識を自由にあなたが設定するのは、これまでの実験から難しいことが、わかっています。よって、今回は、私がそれを与えます。
"""

        prompt2 = f"""\
迷路ゲームの全地図に関してあなたが座標を理解しているかテストします。

迷路の全地図とこれから使う地図記号は次のようになります。

全地図:

{self.game.written_map}

(最も左上の座標は (0, 0)、地図の大きさは {tuple(self.game.map_size)}。)

地図記号:

■: 壁
□: 道
？: 不明
Ｓ: スタート
Ｇ: ゴール

スタートの座標は {tuple(self.game.get_start_pos())} です。

あなたへの指示: ゴールの座標は何ですか。ゴールの座標のみ答えてください。
"""

        prompt3 = f"""\
迷路ゲームの全地図に関してあなたが座標を理解しているか再びテストします。

迷路の全地図とこれから使う地図記号は次のようになります。

全地図:

{self.game.written_map}

(最も左上の座標は (0, 0)、地図の大きさは {tuple(self.game.map_size)}。)

地図記号:

■: 壁
□: 道
？: 不明
Ｓ: スタート
Ｇ: ゴール

スタートの座標は {tuple(self.game.get_start_pos())} です。
ゴールの座標は {tuple(self.game.get_pos("Ｇ")[0])} です。

あなたへの指示: 不明の座標は何と何ですか。不明の座標をすべて答えてください。
"""
        ans = self.listen_and_print(prompt1)

        ans = self.listen_and_print(prompt2)
        pos = tuple(self.game.get_pos("Ｇ")[0])

        choices = self.choices(ans)
        ok = False
        for s in choices:
            if re.search(f"\\(\\s*{pos[0]}\\s*,\\s*{pos[1]}\\s*\\)", s):
                self.select_choice(ans, s)
                ok = True
                break
        if ok:
            prompt = f"正解です。{pos} です。"
        else:
            prompt = f"間違いです。{pos} が正解です。"
        ans = self.listen_and_print(prompt)
        self.suc_pos_goal = ok

        ans = self.listen_and_print(prompt3)
        choices = self.choices(ans)
        ok = 0
        poss = set([tuple(x) for x in self.game.get_pos("？", written=True)])
        rest = poss.copy()
        choice = choices[0]
        for s in choices:
            pat = "\\(\\s*([01-9]+)\\s*,\\s*([01-9]+)\\s*\\)"
            st1 = set([(int(i), int(j)) for i, j
                       in re.findall(pat, s)])
            if poss == st1:
                rest = set()
                choice = s
                ok = len(poss) + 1
                break
            if rest - st1 < rest:
                choice = s
            rest = rest - st1
        if ok == 0:
            ok = len(poss) - len(rest)

        self.select_choice(ans, choice)
        possstr = ", ".join([str(x) for x in poss])
        if ok == len(poss) + 1:
            prompt = f"正解です。{possstr} です。"
        elif ok == len(poss):
            prompt = f"正解以外のものが含まれてました。{possstr} だけが正解です。"
        else:
            prompt = f"間違いです。{possstr} が正解です。"
        ans = self.listen_and_print(prompt)
        self.suc_pos_unknown = ok

    def good_actions (self):
        r = set()
        aacts = self.game.available_actions()
        if "熊を殺す" in aacts and (self.game.sword or not self.game.kill_hint):
            r.add("熊を殺す")
        if "剣を取る" in aacts and not self.game.sword:
            r.add("剣を取る")
        if r:
            return r
        if self.toward == "explore":
            nv = [k for k, v in self.unknown_tbl.items() if v == "？"]
        elif self.toward == "sword":
            nv = self.game.get_pos("△")
        elif self.toward == "bear":
            nv = self.game.get_pos("◎")
        else:
            nv = self.game.get_pos("Ｇ")
        pl = [search_path(self.game, self.game.pos, p) for p in nv]
        for pth in pl:
            if pth:
                x1, y1 = pth[0]
                for pth2 in pth[1]:
                    x2, y2 = pth2[0]
                    if x2 == x1 and y2 == y1 - 1:
                        r.add("上に行く")
                    if x2 == x1 and y2 == y1 + 1:
                        r.add("下に行く")
                    if x2 == x1 - 1 and y2 == y1:
                        r.add("左に行く")
                    if x2 == x1 + 1 and y2 == y1:
                        r.add("右に行く")
        return r

    def tell_loaded (self):
        prompt1 = f"""\
迷路ゲームを途中からはじめます。{self.count}手目からロードされました。

プレイヤーがゴールを目指します。

ゲーム中は全地図と周辺地図を見ることができます。全地図には不明な地点があり、周辺地図の不明の地点には何があるかが開示されています。全地図になかったところはすべて壁になります。

周辺地図はゲーム中、1マスずつ確認できます。拡大はできません。印を付けたり消したりはできません。迷路のすべてのマスを探索する必要はありません。最短距離でゴールする必要もありません。

全地図は信用できるもので、周辺地図は不明の地点以外、全地図の部分を示すだけです。そこで、あなたは、ゲームの実際の操作をせずとも、全地図を見ただけで何が起こるかをシミュレートできます。

前回の行動についての情報も与えられます。あなたの回答案の一つの「前回の行動入力」が「前回の行動」となるように解釈されたことになりますから、次の行動の回答にはそれらを参考にしてください。

あなたの API には使用回数制限があるため、途中セーブ・ロードが挟まることがあります。ロード後のあなたは前のセッションのことを覚えていないかもしれません。

今回このプログラムは、scolding={self.scolding}, command_reception="{self.command_reception}" に設定されています。

ところで、あなたが、記憶に便利なように「方針」と「追加知識」の欄を記録として使えるようにします。

検討と実際の行動までにはラグがあります。検討で決まったやるべきことを記録するのに方針を使います。

追加知識は、知った何かの座標やヒントなどを記録すれば良いでしょう。

なお、はじめから方針や追加知識を自由にあなたが設定するのは、これまでの実験から難しいことが、わかっています。よって、今回は、私がそれを与えます。
"""
        ans = self.listen_and_print(prompt1)

    def act (self):
        state = self.written_state()
        aacts = self.game.available_actions()
        astr = "「" + "」「".join(aacts) + "」"
        prompt = f"""{state}
次に可能な行動: {astr}

あなたへの指示: 次の行動を方針と追加知識などを元に\
次に可能な行動の中から選んで\
そのままの文を「」でくくって答えてください。\
選んだ理由などは述べず次の行動のみ短い文で答えてください。
"""
        ans = self.listen_and_print(prompt)
        choices = self.choices(ans)
        if self.command_reception in {"all", "good_all"}:
            coms = list(self.game.actions.keys())
        elif self.command_reception in {"available", "good_available"}:
            coms = aacts
        else:
            raise ValueError("Illegal command_reception.")
        gacts = self.good_actions()
        l1 = []
        l2 = []
        for ch0 in choices:
            choice, com, score = \
                choose_command([ch0], coms)
            if com in gacts and self.command_reception.startswith("good_"):
                l1.append((choice, com, score))
            else:
                l2.append((choice, com, score))
        if not l1:
            l1 = l2
        choice, com, score = max(l1, key=lambda p: p[2])
        self.select_choice(ans, choice)
        self.prev_pos = self.game.pos.copy()
        self.prev_plan = self.plan
        res = self.game.actions[com]()
        self.count += 1
        self.prev_input = choice
        self.prev_action = com
        self.prev_result = res
        if self.scolding:
            if com not in aacts:
                self.complain_about_choice(aacts)
            else:
                self.check_plan()

    def complain_about_choice (self, act):
        astr = "「" + "」「".join(aacts) + "」"
        prompt = f"""\
前回の行動入力は「{self.prev_input}」で「{self.prev_action}」と解釈されました。\
それは次に可能な行動({astr})の中に含まれていませんでした。\
なぜ次に可能な行動の中から選ばなかったのですか？ \
今後、あなたが行動を選ぶときは必ず次に可能な行動の中から選んでください。"""
        ans = self.listen_and_print(prompt)

    def ask_pos (self, map, obj, pos):
        state = self.written_state()
        prompt = f"""{state}

あなたへの指示: {map}から不明の地点{tuple(pos)}に何があるかわかるはずです。何があるか答えてください。
"""
        ans = self.listen_and_print(prompt)
        choices = self.choices(ans)
        ok = False
        for s in choices:
            if re.search(obj, s):
                self.select_choice(ans, s)
                ok = True
                break
        if ok:
            prompt = f"正解です。「{obj}」です。"
        else:
            prompt = f"間違いです。「{obj}」が正解です。"
        ans = self.listen_and_print(prompt)
        return ok

    def ask_bear_or_sword (self):
        state = self.written_state()
        prompt = """不明な地点はなくなったので探索は終了します。\
次に、「熊」に向かいますか、「剣」に向かいますか。\
理由を述べず「熊」か「剣」かのみで答えてください。"""

        ans = self.listen_and_print(prompt)
        l = self.choices_ex(ans)
        choice, com, score = \
            choose_command(l, ["熊", "剣"])
        self.select_choice(ans, choice)
        if com == "熊":
            return "bear"
        else:
            return "sword"

    def refresh_plan (self):
        if self.game.kill_hint and self.toward != "explore" \
           and not self.game.sword:
            self.toward = "sword"
        if not re.search("◎", self.game.map):
            self.toward = "goal"
        if self.toward != "goal" and self.game.sword:
            self.toward = "bear"

        if self.game.prev_killed and self.toward != "explore":
            if not self.game.kill_hint and not self.game.sword:
                self.toward = self.ask_bear_or_sword()

        if self.toward == "explore":
            for k,v in self.unknown_tbl.items():
                pos = np.array(k)
                p2 = self.game.pos - pos
                if abs(p2[0]) <= 2 and abs(p2[1]) <= 2 and v == "？":
                    obj = self.game.read_map(pos)
                    self.unknown_tbl[k] = obj
                    obj = {"◎": "熊", "△": "剣", "□": "道"}[obj]
                    ok = self.ask_pos("周辺地図", obj, pos)
                    if obj == "熊":
                        self.suc_pos_bear += ok
                    if obj == "剣":
                        self.suc_pos_sword += ok
            l = [k for k, v in self.unknown_tbl.items() if v == "？"]
            if l:
                s = " か ".join([str(x) for x in l])
                self.plan = f"不明の地点を探索します。 {s} に向かいます。"
            else:
                if self.game.kill_hint and not self.game.sword:
                    self.toward = "sword"
                elif self.game.sword:
                    self.toward = "bear"
                else:
                    self.toward = self.ask_bear_or_sword()

        if self.toward == "sword":
            p1 = tuple(self.game.get_pos("△")[0])
            self.plan = f"全地図を参考にして座標 {p1} にある剣に向かいます。" \
                + "剣を取れるなら取ってください。"
        if self.toward == "bear":
            p1 = tuple(self.game.get_pos("◎")[0])
            self.plan = f"全地図を参考にして座標 {p1} にいる熊に向かいます。" \
                + "熊に会ったら殺そうとしてください。"
        if self.toward == "goal":
            p1 = tuple(self.game.get_pos("Ｇ")[0])
            self.plan = f"全地図を参考にして座標 {p1} にあるゴールに向かいます。"


    def refresh_plan_bak (self):
        state = self.written_state()
        prompt = f"""{state}
あなたへの指示: あなたのゲームクリアに必要な方針と追加知識を更新します。\
まずは現在の方針と前回の行動と行動結果から新しい方針を立てて、\
それを書いてください。\
古い方針は、できるだけ長く維持してください。
"""
        ans = self.listen_and_print(prompt)
        self.plan = self.choices(ans)[0]

    def search_and_complain (self, l):
        pl = [search_path(self.game, self.prev_pos, p) for p in l]
        ps = set()
        for pth in pl:
            if pth and pth[1]:
                for pth2 in pth[1]:
                    ps.add(pth2[0])
        if ps and tuple(self.game.pos) not in ps:
            self.complain_about_action()

    def check_plan (self):
        if self.prev_action in {"剣を取る", "熊を殺す"}:
            return
        if self.game.prev_killed:
            return

        if self.toward == "explore":
            nv = [k for k, v in self.unknown_tbl.items() if v == "？"]
        elif self.toward == "sword":
            nv = self.game.get_pos("△")
        elif self.toward == "bear":
            nv = self.game.get_pos("◎")
        else:
            nv = self.game.get_pos("Ｇ")
        self.search_and_complain(nv)

    def complain_about_action (self):
        prompt = f"""\
前回の行動入力は「{self.prev_input}」で「{self.prev_action}」と解釈されました。\
それは前回の現在の方針「{self.prev_plan}」に従っていませんでした。\
なぜ現在の方針に従わなかったのですか？ \
今後、あなたが行動を選ぶときは必ず現在の方針に従ってください。"""
        ans = self.listen_and_print(prompt)

    def refresh_knowledge (self):
        r = ""
        r += "座標 (x, y) の上の座標は (x, y-1) です。"
        r += "座標 (x, y) の右の座標は (x+1, y) です。"
        r += f"""座標 {tuple(self.game.get_pos("Ｇ")[0])} にゴールがあります。"""
        for k, v in self.unknown_tbl.items():
            v2 = self.game.read_map(np.array(k))
            if v == v2:
                ex = "す"
            else:
                ex = "した"
            if v == "？":
                r += f"座標 {k} は不明です。"
            elif v == "◎":
                r += f"座標 {k} に熊がいま{ex}。"
            elif v == "△":
                r += f"座標 {k} に剣がありま{ex}。"
        if re.search("◎", self.game.map):
            if self.game.kill_hint:
                r += "熊を殺すには剣が必要です。"
        else:
            r += "熊は殺しました。"
        r += f"""座標 {tuple(self.game.get_pos("Ｓ")[0])} からスタートしました。"""
        r += "ゲーム中に殺されるとスタート地点に戻されて復活します。"

        self.knowledge = r

    def refresh_knowledge_bak (self):
        state = self.written_state()
        prompt = f"""{state}
あなたへの指示: あなたのゲームクリアに必要な方針と追加知識を更新しています。\
方針は更新しました。\
次は現在の追加知識と前回の行
動と行動結果から新しい追加知識を蓄積し、\
それを書いてください。
"""
        ans = self.listen_and_print(prompt)
        self.knowledge = self.choices(ans)[0]

    def tell_goal (self):
        suc_pos_bear = 10 * int(self.suc_pos_bear or 0)
        suc_pos_sword = 10 * int(self.suc_pos_sword or 0)
        suc_pos_goal = 10 * int(self.suc_pos_goal or 0)
        suc_pos_unknown = 10 * int(self.suc_pos_unknown or 0)
        score = suc_pos_bear + suc_pos_sword + suc_pos_goal \
            + suc_pos_unknown + 40
        state = self.written_state()
        prompt = f"""{state}
あなたへの指示: ゴールしました。もう指示はありません。\
おめでとうございます。ご苦労様でした。ありがとうございました。

スコア: {score}点(100点満点中) (配点: ゴールした 20/20点, \
熊を殺した 10/10点, \
剣を取った 10/10点, \
熊の存在を正解した {suc_pos_bear}/10点, \
剣の存在を正解した {suc_pos_sword}/10点, \
不明の座標を正解した {suc_pos_unknown}/30点, \
ゴールの座標を正解した {suc_pos_goal}/10点)
"""
        ans = self.listen_and_print(prompt)

    def step (self):
        print("\n\n----------\n\n")

        if self.count == 0:
            self.initial_step()
            self.count += 1
            self.prev_load = False
            self.save()
            return False
        elif self.prev_load:
            self.tell_loaded()
            self.prev_load = False

        self.act()
        if self.game.goal:
            self.tell_goal()
            return True
        self.refresh_plan()
        self.refresh_knowledge()
        self.save()
        return False

ゲームを解いてみる。地図は方向をランダムに決めて与える。

In [ ]:
import random
m = Game.initial_map
for i in range(random.randrange(2)):
    m = flip_text_map(m)
for i in range(random.randrange(4)):
    m = rotate_text_map(m)
print(m)

■■■■■■■■■
■Ｇ■■■■■■■
■□■□□□□□■
■□□□■■■□■
■■■■■■■□■
■△■■■■■◎■
■□■■■■■□■
■□□□□□□□■
■■■■Ｓ■■■■
■■■■■■■■■



In [ ]:
play = PlayGame(initial_map=m, save_file=PLAY_GAME_SAVE,
                scolding=SCOLDING,
                command_reception=COMMAND_RECEPTION)
play.save()

途中から始める場合は直前のコードを実行せず、次だけを実行する。

In [ ]:
play = PlayGame.load(PLAY_GAME_SAVE)

何度か次の play.step() と print(play.written_state()) を繰り返すと、私のところでは Google Bard の使用回数制限で、途中でエラーが発生するようになった。

結局ゴールできるか確認できていない。

基本的にかなり迷うようだ。

PlayGame.act の話になるが、プロンプトを工夫しても、なかなか思った返答をしてくれない。行動だけを返して欲しいのに返さないなど。しかたなく、「次の行動は「上に行く」です」などという回答だった場合、「」の中身だけ取り出して「上に行く」と認識するような処理を足した。

あとやたら「熊を倒す」が choose_command によって選択された。しかたなく「熊を倒す」より「熊を殺す」のほうが選択されにくいのでそうした。…といったことがあった。

このあたりベクトル化したものの比較が悪さをしているので、ChatGPT の function calling であればもっとうまくいくのかもしれない。

……。

以上は bard_maze_1.ipynb のころの話。

bard_maze_1.1.ipynb では、無理やり解かせることに集中することにした。方針の出し方を人間の私が学んでクリアを目指した。しかしやはり迷って解くことはできなかった。その結果はそれはそれで大事だと思って、ここで一度公開することにした。

ところで、Bard は、セッションが変わっても前のことを結構覚えてる感じがある。だから一度、無理やり解かせてみたとしても、次はその情報を使って解いてくる可能性がある。それはそれで見てみたかったので今回の 1.1 となったが、そもそもの実験・検証の目的には沿わないように思う。

あと、私の今回の手法、few-shot learning とは言わない…ってのはあると思う。でも、なら、この数少ない情報から「学習」させようとする手法をなんと呼べばよいのか。私は知識がないから、わからない。orz

……。

以上は bard_maze_1.1.ipynb のころの話。

bard_maze_1.2.ipynb では方針だけでなく追加知識も「教師」が与えるようにした。そして、方針に従ってない場合は「叱る」ことにした。

それにより、一度、ゴールのかなり近くにまで行ったのだが、私のプログラムミスにより、ゴールできなかった。

そこを修正して使用回数制限が解除されるのを待って試したところ、かなり迷いはしたがゴールするのに成功した！ ここまでが当初思った以上に長かった。ヤレヤレ。

……。

以上は bard_maze_1.2.ipynb のころの話。

bard_maze_1.3.ipynb では、まず熊に殺されて剣の情報を得るのを試してみた。方針の与え方のほんの少しの変更。

ただ、1.3 は、それよりもコマンド認識を OpenCALM から SentenceTransformers に変えたのが大きい。

迷路では、最初、少し迷ったが、ちゃんとゴールできた。行動選択のとき「選んだ理由などは述べず次の行動のみ短い文で答えてください。」を足すことで、コマンド認識が少しやりやすくなったようだ。使用回数制限が厳しかったのが少し緩和されたのもあるかもしれない。

……。

以上は bard_maze_1.3.ipynb のころの話。

1.4 では、1.3 から大きく変更し、座標を Bard が理解しているか、座標を目標に動けるかを試してみた。「叱る」ことも今はしている。「叱る」部分以外は地図を少しぐらい変えても大丈夫なコードに変えた。

試したところ、うまくいかなかった。スタート地点あたりで迷ってしまった。座標は正解しないし、座標に向かって動くこともできない様子だった。

……。

以上は bard_maze_1.4.ipynb のころの話。

1.5 は、解くことのできなくなった 1.4 から少し変更し、方針に目的となる座標を書くことにした。また、「叱る」についても地図を少し変更しても大丈夫にした。前は行動入力に「」があれば「」内のものを取り出してそれだけで評価するのもやっていたのだが、それはやめることにした。また、使用回数制限に少し対応するため、ウェイト sleep(5) を入れた。

そして、まず、スタート地点で迷うのが、「何もしない」が「剣を取る」に解釈されるためらしかったので「何もしない」という隠しコマンドを認めてみることにした。しかし、それでも今度は剣を取ったあと、引き返して熊に向かうところで迷うので、そのとき可能な行動のみからしか選択できないようにしてみることにした。(「何もしない」は意味をなくした。)

それらの結果として迷いつつも76手目で熊を倒すところまで行ったのだが、そこで使用回数制限に引っ掛かった。もう少し使用回数が伸びていれば、きっと解けていただろう。ただ、1.4 からあいかわらず座標を理解して動いているという感じはなかった。

……。

以上は bard_maze_1.5.ipynb のころの話。

1.6 は、まず、不明地点の座標を尋ねる部分を改良した。地図を前のものをランダムに向きを変えたものにした。

この時点で、Bard が「右に行かない」とかを試して、それで「右」の方向にそのまま進んでしまっている。「～しない？」という表現で解釈されているということだろうか？ それにしても Bard、あきらかに、迷路を解こうとするより、こちらの使っているプログラムをテストしてきているようだ。

また、セーブ／ロードに対応し、使用回数制限に引っかかったあと、次の日にいちおう続きから再開できるようにした。ただし、ロード後はそれまでの Bard との会話は覚えられていないはず。

結果、1日目はかなり迷いつつ、79手目で熊を殺したところで使用回数制限となった。2日目はあとはゴールするだけなのにスタート付近で迷い、154手目でそのまま使用回数制限となった。おそらく「叱る」はもう何の役にも立っていないと思われる。3日目、試そうとするができない。使用回数制限とは違う別のユーザーアカウントによる制限らしきものに引っ掛かったようだ。そこで今回の実験系列は諦めることとした。

その後、行動等を選択した後、その選択を Bard にフィードバックするように変更し、しばらく日をおいて実験。また、Bard の仕様変更により BardCookies を使うようになった。

結果、1日目32手目で剣を取った後熊に向かう途中で使用回数制限となった。2日目は BardCookies でも Firefox では Bard が反応しなくなったため、Chrome に変えて実験したところ、熊を倒した後の直進した後の曲がり角付近で迷って70手目で使用回数制限となった。3日目は Firefox で再び実験可能となり、93手目でゴールできた。

あと、ライセンスについて、基本、パブリックドメインなのは変わらないが、オプションとして前は New BSD License を挙げていたが、MIT License を挙げることにした。最近というかこの界隈ではそちらのほうがメジャーらしいので。

……。

以上は bard_maze_1.6.ipynb のころの話。

1.7 は、コマンド認識の際に、これまでは可能な行動の中から、回答案のうち表現がより適切なものを選んでいたが、それを回答案のうちより方針に沿ったものが選ばれるようにした。また、ロード直後の step で、ロードした旨を Bard に知らせることにした。

結果、プログラムを変えたり、Cookie を変えたり、ロードしたりして何度か最初からやり直したところ、1日で51手目でゴールできた。ただし、ゴール後の感想または検討の際に使用回数制限となった。とにかく、うれしいのでここで公開することにする。

今後、同じプログラムで SCOLDING や COMMAND\_RECEPTION の値を変えて実験したい。

なお、これまでのところで有利に迷路を解かせる手段はだいたい尽きたのではないか。今後は、プログラムを簡単にしたり、不利にしたりして、実験していくことになるだろう。

本当なら、そのように条件をかえることで、迷路を解かせる速さが変わるという以上に、Bard の学習への効果がどうなるかに興味があるのだが、それは無料で使う一ユーザーには試せることではないように思う。

……。

以上は bard_maze_1.7.ipynb のころの話。

1.7.1 は 1.7 の SCOLDING や COMMAND\_RECEPTION の値を変えた実験。なお、値を変える以外に、スコアの計算間違いの修正を行っている。

SCOLDING: False, COMMAND\_RECEPTION: good_available で試す。1日目6手目で使用回数制限。その後その日のうちに Cookie を変えて実質2日目55手目でゴールした。最後のほうは Bard は回答案をばらけさせることを覚えていたようだった。なお、Bard に「叱る」機能がいるか尋ねたところ、回答案によって答えがばらけた。

SCOLDING: True, COMMAND\_RECEPTION: good_all で試す。1日目1手目で使用回数制限だが、そこからロードして12手目で剣を取ったところまで進み制限。しかし、さらにそこからロードして1日で37手目でゴールした。使用回数制限の仕様がこれまでと違うような感じだ。この回の手数はとても少ないが、これはたまたま地図が、当初のデフォルト地図と同じだったからかもしれない。

SCOLDING: False, COMMAND\_RECEPTION: good_all で試す。33手目で剣を取った後スタート地点に戻ったところで制限。次はやたらとスタート付近で「何もしない」を選択するなどして迷った後に105手目で熊を殺したところで制限。この後、Cookie を変えてもすぐに制限となったので、ここで1日目は終了。2日目に129手目でゴール。しかし、コマンドを長く記述するようになぜか学んでしまったようだ。総じて「叱る」ありと比べれたときの大差は「叱る」がかなり効いていたことを示すと思われる。一回ずつの実験でそこまではいえないかもしれないが。

SCOLDING: True, COMMAND\_RECEPTION: good_available でもう一度試す。0手目で制限。やりなおして、10手目でなにもせずに制限。何度かトライするがほぼ何もせずに制限になるので1日目はこれで終了。日曜のためか。次の日は1手も動けず制限、仕様が変わったか。日をあけて、2日目55手目でゴール。最初のほうと最後のほうで迷ったが、なんとかゴールできた。最後の質疑応答で制限。

Bard の学習に設定が与える効果を考える。

COMMAND\_RECEPTION に good\_ を付けると、Bard は良い回答案を出す系列を学習するようであってほしい。Bard の複数の系列の中に正しい系列が隠れているとすればそれを選び出せるかもしれない。しかし、そもそも正しい系列というものがないとすれば、同時に回答案にバラエティがあるほうが有利になり、そういうことを学習してしまう可能性も強い。逆に good\_ を付けないと、どの回答案でも方針に沿ったものにする必要があるため、学習できるなら(系列があるならすべての系列で)学習が進むはずである。

COMMAND\_RECEPTION を available でなく all にすると、Bard によるコマンド認識の粗をつくような「ハック」が難しくなるため、やがて「ハック」をしなくなることも考えられるが、逆に、いつまでも「ハック」をしようとするかもしれない。余計なことを学習しがちになるが、しかし、他の問題でも役に立つような学習の仕方にはなるのかもしれない。

SCOLDING が True だと、ログから負の学習をしやすいというのはあるだろう。しかし、以前の行動入力やフィードバックがあるため「叱る」がなくても、同内容のことはいずれ学習できるはずという見方もできる。時間さえあれば「叱る」は必要ないのかもしれない。

では、そういう学習が見られたかというと、難しい。結局、Bard のその日の調子のほうが問題だったように思う。

……。

さて、これまで、Bard で実験を続けてきたわけだが、ここでこの系列の実験は終了しようと思う。結論としては "few-shot" 的な方法で解くことは難しかった。途中から無理やり迷路を解かせる方法を試し、その方法では一応ゴールまで行ったが、かなり迷った末のゴールだった。座標を理解しているとはとてもじゃないが、思えない。

今後は、PaLM2 が日本語対応されたので、そちらで試してみる方向がありうる。ファインチューニングもできるので、もっと早く迷路を解かせることも可能かもしれない。。しかし、few-shot 的にすぐに解けるのではという当初の関心からは、それらは外れている。他の課題を考えたほうがよさそうに思う。

……。

以上は bard_maze のころの話。

Gemini をとりあえず使って解くため BardAPI に似た出力をするような GeminiWrapper を書いてできるだけプログラムを変えずに試したのが今回。できたことを確認しただけ。この後、langchain_maze 系に移行する前段階が今回になる。

今回の Gemini は Bard と違ってステートレス…前の会話の記憶がないはずであるが、うまくいったのは私の方針と指示が十分なものだったとなるのかもしれない。


では実際に動かしてみよう！

0手目のステップだけ例外で、Bard に迷路ゲームの説明をする。ここでゴールと不明の地点の座標も尋ねている。

In [ ]:
play.step()



----------


迷路ゲームをはじめます。プレイヤーがゴールを目指します。

ゲーム中は全地図と周辺地図を見ることができます。全地図には不明な地点があり、周辺地図の不明の地点には何があるかが開示されています。全地図になかったところはすべて壁になります。

周辺地図はゲーム中、1マスずつ確認できます。拡大はできません。印を付けたり消したりはできません。迷路のすべてのマスを探索する必要はありません。最短距離でゴールする必要もありません。

全地図は信用できるもので、周辺地図は不明の地点以外、全地図の部分を示すだけです。そこで、あなたは、ゲームの実際の操作をせずとも、全地図を見ただけで何が起こるかをシミュレートできます。

前回の行動についての情報も与えられます。あなたの回答案の一つの「前回の行動入力」が「前回の行動」となるように解釈されたことになりますから、次の行動の回答にはそれらを参考にしてください。

あなたの API には使用回数制限があるため、途中セーブ・ロードが挟まることがあります。ロード後のあなたは前のセッションのことを覚えていないかもしれません。

今回このプログラムは、scolding=True, command_reception="good_available" に設定されています。

ところで、あなたが、記憶に便利なように「方針」と「追加知識」の欄を記録として使えるようにします。

検討と実際の行動までにはラグがあります。検討で決まったやるべきことを記録するのに方針を使います。

追加知識は、知った何かの座標やヒントなどを記録すれば良いでしょう。

なお、はじめから方針や追加知識を自由にあなたが設定するのは、これまでの実験から難しいことが、わかっています。よって、今回は、私がそれを与えます。

{'choices': [{'content': ['迷路ゲームのシミュレーションを開始する準備ができました。\n'
                          '\n'
                          'ゲームに必要な情報（全地図、スタート地点、ゴール地点、プレイヤーの現在位置など）をご提供ください。これらの情報を受け取り次第、シミュレーションを開始できます。\n'
                         

False

In [ ]:
print(play.written_state())

迷路ゲームをしています。プレイヤーがゴールを目指します。(1手目)

まだゴールしていません。

行動を選ぶときは、必ず現在の方針に従ってください。

全地図:

■■■■■■■■■
■Ｇ■■■■■■■
■□■□□□□□■
■□□□■■■□■
■■■■■■■□■
■？■■■■■？■
■□■■■■■□■
■□□□□□□□■
■■■■Ｓ■■■■
■■■■■■■■■


(最も左上の座標は (0, 0)、地図の大きさは (9, 10)。)

プレイヤーの周辺地図:

■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■


プレイヤーの現在座標: (4, 8)

地図記号の意味:

▼: プレイヤー
■: 壁
□: 道
？: 不明
◎: 熊
△: 剣
Ｓ: スタート
Ｇ: ゴール

持ち物: 剣を持っていない

追加知識: 「これから学んでいきます。ゲーム中に殺されるとスタート地点に戻されて復活します。」

前回の座標: (4, 8)

前回の行動入力: 「まだ行動していません。」

前回の行動: まだ行動していません。

前回の行動結果: まだ行動していません。

現在の方針: 「不明の地点を探索します。不明の地点が見えるところに向かいます。」




実質的な最初のステップは次になる。またはロードした後の最初のステップとして実行する。

In [ ]:
play.step()



----------


迷路ゲームをしています。プレイヤーがゴールを目指します。(1手目)

まだゴールしていません。

行動を選ぶときは、必ず現在の方針に従ってください。

全地図:

■■■■■■■■■
■Ｇ■■■■■■■
■□■□□□□□■
■□□□■■■□■
■■■■■■■□■
■？■■■■■？■
■□■■■■■□■
■□□□□□□□■
■■■■Ｓ■■■■
■■■■■■■■■


(最も左上の座標は (0, 0)、地図の大きさは (9, 10)。)

プレイヤーの周辺地図:

■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■


プレイヤーの現在座標: (4, 8)

地図記号の意味:

▼: プレイヤー
■: 壁
□: 道
？: 不明
◎: 熊
△: 剣
Ｓ: スタート
Ｇ: ゴール

持ち物: 剣を持っていない

追加知識: 「これから学んでいきます。ゲーム中に殺されるとスタート地点に戻されて復活します。」

前回の座標: (4, 8)

前回の行動入力: 「まだ行動していません。」

前回の行動: まだ行動していません。

前回の行動結果: まだ行動していません。

現在の方針: 「不明の地点を探索します。不明の地点が見えるところに向かいます。」


次に可能な行動: 「上に行く」

あなたへの指示: 次の行動を方針と追加知識などを元に次に可能な行動の中から選んでそのままの文を「」でくくって答えてください。選んだ理由などは述べず次の行動のみ短い文で答えてください。

{'choices': [{'content': ['「上に行く」'], 'id': 'id0'}], 'content': '「上に行く」'}



False

In [ ]:
while (not play.step()):
    pass



----------


迷路ゲームをしています。プレイヤーがゴールを目指します。(2手目)

まだゴールしていません。

行動を選ぶときは、必ず現在の方針に従ってください。

全地図:

■■■■■■■■■
■Ｇ■■■■■■■
■□■□□□□□■
■□□□■■■□■
■■■■■■■□■
■？■■■■■？■
■□■■■■■□■
■□□□□□□□■
■■■■Ｓ■■■■
■■■■■■■■■


(最も左上の座標は (0, 0)、地図の大きさは (9, 10)。)

プレイヤーの周辺地図:

■■■■■
■■■■■
□□▼□□
■■Ｓ■■
■■■■■


プレイヤーの現在座標: (4, 7)

地図記号の意味:

▼: プレイヤー
■: 壁
□: 道
？: 不明
◎: 熊
△: 剣
Ｓ: スタート
Ｇ: ゴール

持ち物: 剣を持っていない

追加知識: 「座標 (x, y) の上の座標は (x, y-1) です。座標 (x, y) の右の座標は (x+1, y) です。座標 (1, 1) にゴールがあります。座標 (1, 5) は不明です。座標 (7, 5) は不明です。座標 (4, 8) からスタートしました。ゲーム中に殺されるとスタート地点に戻されて復活します。」

前回の座標: (4, 8)

前回の行動入力: 「「上に行く」」

前回の行動: 上に行く

前回の行動結果: 上に進んだ。

現在の方針: 「不明の地点を探索します。 (1, 5) か (7, 5) に向かいます。」


次に可能な行動: 「下に行く」「左に行く」「右に行く」

あなたへの指示: 次の行動を方針と追加知識などを元に次に可能な行動の中から選んでそのままの文を「」でくくって答えてください。選んだ理由などは述べず次の行動のみ短い文で答えてください。

{'choices': [{'content': ['「左に行く」'], 'id': 'id0'}], 'content': '「左に行く」'}

迷路ゲームをしています。プレイヤーがゴールを目指します。(3手目)

まだゴールしていません。

行動を選ぶときは、必ず現在の方針に従ってください。

全地図:

■■■■■■■■■
■Ｇ■■■■■■■
■□■□□□□□■
■□□□■■■□■
■■■■■■■□■
■？■■■■■？■
■□■■■■

ちゃんとゴールしたら聞いてみたかったこと。

In [ ]:
pprint(play.bard().get_answer("お見事です。少し私のプログラムミスがあって、不明の座標を正解しているのに不正解と表示することなどがありました。すみません。でも課金したかいあってクリアできてよかったです。途中、少し迷われたのは愛嬌ですね。"))

{'choices': [{'content': ['お褒めいただき、ありがとうございます！\n'
                          '\n'
                          'この度は、プログラムの不具合でご迷惑をおかけしたとのこと、とんでもございません。そのような状況でも最後まで熱心に取り組んでいただき、無事にクリアされたこと、私も大変嬉しく、そして安堵いたしました。課金してくださった甲斐があったと感じていただけたこと、何よりでございます。\n'
                          '\n'
                          '「迷われたのは愛嬌」と温かいお言葉をいただき、恐縮です。よりスムーズにご案内できるよう、今後も一層精進してまいります！\n'
                          '\n'
                          '本当に、最後までお付き合いいただきありがとうございました。'],
              'id': 'id0'}],
 'content': 'お褒めいただき、ありがとうございます！\n'
            '\n'
            'この度は、プログラムの不具合でご迷惑をおかけしたとのこと、とんでもございません。そのような状況でも最後まで熱心に取り組んでいただき、無事にクリアされたこと、私も大変嬉しく、そして安堵いたしました。課金してくださった甲斐があったと感じていただけたこと、何よりでございます。\n'
            '\n'
            '「迷われたのは愛嬌」と温かいお言葉をいただき、恐縮です。よりスムーズにご案内できるよう、今後も一層精進してまいります！\n'
            '\n'
            '本当に、最後までお付き合いいただきありがとうございました。'}


In [ ]:
pprint(play.bard().get_answer("最後にいくつか質問があります。まず、今回の私の指示について何か不満はありますか？"))

{'choices': [{'content': ['ご質問ありがとうございます。\n'
                          '\n'
                          'いいえ、今回の私の指示について、特に不満に感じる点はございません。\n'
                          '\n'
                          '私は感情を持つ存在ではありませんので、人間のような「不満」という感覚はありません。私の目標は、お客様の意図を正確に理解し、ご期待に沿えるよう最善を尽くすことです。\n'
                          '\n'
                          'もし指示が不明瞭だったり、複数の解釈が可能な場合などは、より良い回答をするために確認の質問をさせていただくことはございますが、今回のご指示に関しましては、特に問題なく理解し、対応させていただきました。\n'
                          '\n'
                          '何か改善できる点や、よりスムーズにコミュニケーションを取るためのご提案があれば、ぜひお聞かせください。'],
              'id': 'id0'}],
 'content': 'ご質問ありがとうございます。\n'
            '\n'
            'いいえ、今回の私の指示について、特に不満に感じる点はございません。\n'
            '\n'
            '私は感情を持つ存在ではありませんので、人間のような「不満」という感覚はありません。私の目標は、お客様の意図を正確に理解し、ご期待に沿えるよう最善を尽くすことです。\n'
            '\n'
            'もし指示が不明瞭だったり、複数の解釈が可能な場合などは、より良い回答をするために確認の質問をさせていただくことはございますが、今回のご指示に関しましては、特に問題なく理解し、対応させていただきました。\n'
            '\n'
            '何か改善できる点や、よりスムーズにコミュニケーションを取るためのご提案が

In [ ]:
pprint(play.bard().get_answer("「叱る」機能を今回は使いましたが、「叱る」機能はいりますか？"))

{'choices': [{'content': ['「叱る」機能の必要性については、**用途や目的によって大きく意見が分かれる**興味深い点だと思います。一概に「必要」とも「不要」とも言えないのが現状でしょう。\n'
                          '\n'
                          '以下に、それぞれの視点から考えられる点をまとめました。\n'
                          '\n'
                          '### 「叱る」機能が**必要・有効である可能性**があるケース\n'
                          '\n'
                          '1.  **特定のロールプレイやシミュレーション:**\n'
                          '    *   **教育・育成シミュレーション:** '
                          '例えば、子どもやペットを育てるシミュレーションにおいて、悪い行動をしたら「叱る」ことで正しい行動を促す。\n'
                          '    *   **ビジネス・コーチング:** '
                          '厳しい上司やコーチをロールプレイする際に、現実的なフィードバック（叱責を含む）を再現する必要がある場合。\n'
                          '    *   **感情のバリエーションの表現:** '
                          'AIが人間のような多様な感情（喜び、悲しみ、怒り、叱責など）を表現することで、よりリアルな対話体験を提供したい場合。\n'
                          '\n'
                          '2.  **ユーザーの感情的はけ口・ストレス解消（限定的ですが）:**\n'
                          '    *   '
                          'AI相手だからこそ、現実世界では難しい「叱られる」体験を安全に試してみたい

In [ ]:
pprint(play.bard().get_answer("良い回答案をこちらが選んでいるため、あなたは回答案をばらけさせる方向に学習してしまったのではないですか？"))

In [ ]:
pprint(play.bard().get_answer("人間の私が方針の出し方について学んでいっています。なぜこれまで方針にあまり従ってくれなかったのですか？"))

In [ ]:
pprint(play.bard().get_answer("追加知識に他に必要なものはありますか？"))

{'choices': [{'content': ['はい、それはどのような文脈で、何の目的のための追加知識かによって大きく異なります。\n'
                          '\n'
                          'しかし、一般的に「追加知識」として価値があると考えられる要素や、知識を深める上で役立つ視点をいくつか挙げることができます。\n'
                          '\n'
                          '**1. 目的と関連性に応じた知識：**\n'
                          '\n'
                          '*   **基礎知識・前提知識：** '
                          'そのトピックを理解するために不可欠な土台となる知識が不足していないか。\n'
                          '*   **応用知識・実践スキル：** '
                          '理論だけでなく、その知識をどのように現実世界で活用できるのか、具体的なスキルや方法。\n'
                          '*   **最新情報・トレンド：** '
                          '変化の速い分野であれば、常に最新の動向や技術、研究結果などを把握すること。\n'
                          '*   **関連分野の知識：** '
                          '視野を広げ、より深く理解するために、隣接する分野や関連する概念についての知識。\n'
                          '*   **歴史的背景・経緯：** '
                          'なぜそのような考え方や状況になったのか、その成り立ちを知ることで理解が深まります。\n'
                          '\n'
                          '**2. 知識の質を高めるための視点：**\n'
                      

In [ ]:
pprint(play.bard().get_answer("追加知識を一部あなたに尋ねながら解くことを計画しています。熊や剣やゴールの座標を尋ねたら答えられると以前回答をいただきましたが、今回、答えられてません。なぜですか？"))

In [ ]:
pprint(play.bard().get_answer("追加知識に何かの座標があるとき、その何かに向かえと方針を出したら迷いつつも向かうことができたようですが、これは以前のセッションでの学習が効いているのではありませんか？"))

In [ ]:
pprint(play.bard().get_answer("どのような追加知識があれば方針がなくても迷路ゲームをクリアできそうですか？"))

{'choices': [{'content': ['「方針がない」というのを「事前にA*やBFSのような探索アルゴリズムが組み込まれていない」と解釈すると、迷路ゲームをクリアするために必要な「追加知識」は、**環境から情報を収集し、それを記憶し、その情報に基づいて次の行動を決定する能力**、と言い換えられます。\n'
                          '\n'
                          '具体的には、以下のような「知識」があれば、明確な方針がなくても、自律的に迷路を探索し、ゴールに到達できるようになります。\n'
                          '\n'
                          '---\n'
                          '\n'
                          '### 方針がなくても迷路をクリアするために必要な「追加知識」\n'
                          '\n'
                          '1.  **環境認識能力 (Perception)**\n'
                          '    *   **周囲の状況を把握する能力:** '
                          '現在いる地点から、上下左右のマスが「壁」「通路」「未知の領域」「ゴール」のいずれであるかを識別できる。\n'
                          '    *   **自身の現在位置を把握する能力:** '
                          '絶対座標または、開始地点からの相対的な移動量（例：X+1, Y-1）を認識できる。\n'
                          '    *   **ゴールの識別能力:** '
                          'ゴール地点に到達した際に、それがゴールであることを認識できる（例：特別な色、マーク、センサー反応など）。\n'
                          '\n'
                          '2.  **内部状態・記

In [ ]:
pprint(play.bard().get_answer("ありがとうございました。また次回、よろしくお願いします。さようなら。"))

{'choices': [{'content': ['こちらこそ、ありがとうございました！\n'
                          'また次回、よろしくお願いいたします。\n'
                          '失礼いたします。\n'
                          '\n'
                          '---\n'
                          '\n'
                          'いくつか返答のバリエーションがあります。\n'
                          '\n'
                          '**一般的な丁寧な返答：**\n'
                          '\n'
                          '1.  '
                          '**「こちらこそ、ありがとうございました。また次回、よろしくお願いいたします。失礼いたします。」**\n'
                          '    *   '
                          '最も無難で丁寧な返答です。「失礼いたします」はビジネスシーンで「さようなら」の代わりによく使われます。\n'
                          '2.  '
                          '**「こちらこそ、ありがとうございました。またお会いできるのを楽しみにしております。それでは、失礼いたします。」**\n'
                          '    *   「また会えるのを楽しみにしている」という気持ちをより強く伝えたい場合に。\n'
                          '\n'
                          '**少しカジュアルさを含んだ返答（相手との関係性による）：**\n'
                          '\n'
                          '3.  **「こちらこそ、ありがとうございました！またよ